## Read data

In [1]:
#China

In [1]:
import pandas as pd
def file_reader(path):
    df=pd.DataFrame()
    chunksize = 10 ** 6
    for chunk in pd.read_csv(path, chunksize=chunksize, index_col=[0]):
        df = df.append(chunk)
    return df

country='China'
d1=file_reader('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/01_Suspend tweet/%s.csv' %(country))
d2=file_reader('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/03_Current %s.csv' %(country))

data=pd.concat([d1,d2], ignore_index=True)

In [2]:
data

,tweet_text,tweet_time,suspend
0,Necessity is the mother of invention,2019-10-18 21:21,True
1,#病毒 Constantly defying the will of the citize...,2020-03-26 05:09,True
2,#肺炎 #病毒 The priceless life should not be the...,2020-03-28 02:37,True
3,"Want to see the walking dead, but feel a littl...",2019-10-30 05:57,True
4,Maybe should call that Kekong? #HongKong #Keki...,2020-01-03 01:39,True
...,...,...,...
44293,@ace_critic @realDonaldTrump @narendramodi The...,2020-07-23 02:48:38,False
44294,RT @F1Bathurst2015: @catherine___c @tyrelle123...,2020-07-23 09:24:57,False
44295,RT @YeOldHongKonger: .\nBioterorist Xi Jinping...,2020-07-23 00:25:19,False
44296,@GauravS101418 @narendramodi Hahaha... So u ex...,2020-07-23 01:57:29,False


## Sentimental Analysis

In [3]:
#data preprocessing

import nltk
from nltk.tokenize import TweetTokenizer
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()
def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import re
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', (word))
        if new_word != '':
            new_words.append(new_word)
    return new_words

import preprocessor as p
data['text']=0
for i,v in enumerate(data['tweet_text']):
    data.loc[i,'text'] = p.clean(str(v))
data['text'] = data['text'].astype(str).str.replace('\d+', '')
data['text'] = data['text'].str.lower()

for i in range(len(data)):
    data.text[i] = lemmatize_text(data.text[i])

data['text'] = data['text'].apply(lambda x: [item for item in x if item not in stop_words])   

new_text=[]
for i in data.text:
    i=remove_punctuation(i)
    new_text.append(i)

data['text']=new_text

In [4]:
data

,tweet_text,tweet_time,suspend,text
0,Necessity is the mother of invention,2019-10-18 21:21,True,"[necessity, mother, invention]"
1,#病毒 Constantly defying the will of the citize...,2020-03-26 05:09,True,"[constantly, defying, citizen, constantly, cre..."
2,#肺炎 #病毒 The priceless life should not be the...,2020-03-28 02:37,True,"[priceless, life, victim, politician, pursuit,..."
3,"Want to see the walking dead, but feel a littl...",2019-10-30 05:57,True,"[want, see, walking, dead, feel, little, scared]"
4,Maybe should call that Kekong? #HongKong #Keki...,2020-01-03 01:39,True,"[maybe, call, kekong]"
...,...,...,...,...
44293,@ace_critic @realDonaldTrump @narendramodi The...,2020-07-23 02:48:38,False,"[occurrence, virus, infection, matter, chinese]"
44294,RT @F1Bathurst2015: @catherine___c @tyrelle123...,2020-07-23 09:24:57,False,"[absolutely, idea, th, real, problem, black, l..."
44295,RT @YeOldHongKonger: .\nBioterorist Xi Jinping...,2020-07-23 00:25:19,False,"[bioterorist, xi, jinpings, totalitarian, disr..."
44296,@GauravS101418 @narendramodi Hahaha... So u ex...,2020-07-23 01:57:29,False,"[hahaha, u, expect, pm, create, panic, society..."


In [5]:
#sentimental analysis
from textblob import TextBlob

def polarity_text(num):
    if num > 0:
        text='positive'
    elif num == 0:
        text='neural'
    elif num <0:
        text='negative'
    return text 

def subject_text(num):
    if num > 0.5:
        text='subjective'
    elif num == 0.5:
        text='neural'
    elif num < 0.5:
        text='objective'
    return text


p=[]
s=[]
pn=[]
sn=[]
for i in data.text:
    i=" ".join(i)
    testimonial = TextBlob(i)
    pn.append(testimonial.sentiment.polarity)
    sn.append(testimonial.sentiment.subjectivity)
    p.append(polarity_text(testimonial.sentiment.polarity))
    s.append(subject_text(testimonial.sentiment.subjectivity))

data['polarity']=p
data['subjectivity']=s
data['polarity_number']=pn
data['subjectivity_number']=sn

In [7]:
data

,tweet_text,tweet_time,suspend,text,polarity,subjectivity,polarity_number,subjectivity_number
0,Necessity is the mother of invention,2019-10-18 21:21,True,"[necessity, mother, invention]",neural,objective,0.000000,0.000000
1,#病毒 Constantly defying the will of the citize...,2020-03-26 05:09,True,"[constantly, defying, citizen, constantly, cre...",negative,objective,-0.150000,0.316667
2,#肺炎 #病毒 The priceless life should not be the...,2020-03-28 02:37,True,"[priceless, life, victim, politician, pursuit,...",positive,objective,0.308333,0.383333
3,"Want to see the walking dead, but feel a littl...",2019-10-30 05:57,True,"[want, see, walking, dead, feel, little, scared]",negative,objective,-0.193750,0.450000
4,Maybe should call that Kekong? #HongKong #Keki...,2020-01-03 01:39,True,"[maybe, call, kekong]",neural,objective,0.000000,0.000000
...,...,...,...,...,...,...,...,...
44293,@ace_critic @realDonaldTrump @narendramodi The...,2020-07-23 02:48:38,False,"[occurrence, virus, infection, matter, chinese]",neural,objective,0.000000,0.000000
44294,RT @F1Bathurst2015: @catherine___c @tyrelle123...,2020-07-23 09:24:57,False,"[absolutely, idea, th, real, problem, black, l...",positive,subjective,0.077778,0.544444
44295,RT @YeOldHongKonger: .\nBioterorist Xi Jinping...,2020-07-23 00:25:19,False,"[bioterorist, xi, jinpings, totalitarian, disr...",neural,objective,0.000000,0.100000
44296,@GauravS101418 @narendramodi Hahaha... So u ex...,2020-07-23 01:57:29,False,"[hahaha, u, expect, pm, create, panic, society...",positive,objective,0.200000,0.400000


In [8]:
data=data.drop(['text'],axis=1)

In [9]:
data

,tweet_text,tweet_time,suspend,polarity,subjectivity,polarity_number,subjectivity_number
0,Necessity is the mother of invention,2019-10-18 21:21,True,neural,objective,0.000000,0.000000
1,#病毒 Constantly defying the will of the citize...,2020-03-26 05:09,True,negative,objective,-0.150000,0.316667
2,#肺炎 #病毒 The priceless life should not be the...,2020-03-28 02:37,True,positive,objective,0.308333,0.383333
3,"Want to see the walking dead, but feel a littl...",2019-10-30 05:57,True,negative,objective,-0.193750,0.450000
4,Maybe should call that Kekong? #HongKong #Keki...,2020-01-03 01:39,True,neural,objective,0.000000,0.000000
...,...,...,...,...,...,...,...
44293,@ace_critic @realDonaldTrump @narendramodi The...,2020-07-23 02:48:38,False,neural,objective,0.000000,0.000000
44294,RT @F1Bathurst2015: @catherine___c @tyrelle123...,2020-07-23 09:24:57,False,positive,subjective,0.077778,0.544444
44295,RT @YeOldHongKonger: .\nBioterorist Xi Jinping...,2020-07-23 00:25:19,False,neural,objective,0.000000,0.100000
44296,@GauravS101418 @narendramodi Hahaha... So u ex...,2020-07-23 01:57:29,False,positive,objective,0.200000,0.400000


In [10]:
data.to_csv('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/04_Tweets/%s.csv' %(country)) 

#

In [36]:
#Russia

In [37]:
country='Russia'
d1=file_reader('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/01_Suspend tweet/%s.csv' %(country))
d2=file_reader('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/03_Current %s.csv' %(country))

data=pd.concat([d1,d2], ignore_index=True)

In [38]:
data['text']=0
import preprocessor as p
for i,v in enumerate(data['tweet_text']):
    data.loc[i,'text'] = p.clean(str(v))
data['text'] = data['text'].astype(str).str.replace('\d+', '')
data['text'] = data['text'].str.lower()

for i in range(len(data)):
    data.text[i] = lemmatize_text(data.text[i])

data['text'] = data['text'].apply(lambda x: [item for item in x if item not in stop_words])   

new_text=[]
for i in data.text:
    i=remove_punctuation(i)
    new_text.append(i)

data['text']=new_text

In [39]:
p=[]
s=[]
pn=[]
sn=[]
for i in data.text:
    i=" ".join(i)
    testimonial = TextBlob(i)
    pn.append(testimonial.sentiment.polarity)
    sn.append(testimonial.sentiment.subjectivity)
    p.append(polarity_text(testimonial.sentiment.polarity))
    s.append(subject_text(testimonial.sentiment.subjectivity))

data['polarity']=p
data['subjectivity']=s
data['polarity_number']=pn
data['subjectivity_number']=sn

In [15]:
data=data.drop(['text'],axis=1)

In [16]:
data.to_csv('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/04_Tweets/%s.csv' %(country)) 

In [17]:
data

,tweet_text,tweet_time,suspend,polarity,subjectivity,polarity_number,subjectivity_number
0,"Barcelona is so sunny and nice at Vmworld, tre...",2013-10-13 10:32,True,positive,subjective,0.400000,0.600000
1,Linear -up and –out scaling on disk and node l...,2013-10-15 12:30,True,positive,objective,0.200000,0.200000
2,open for a surprise https://t.co/YrU8x3AhGC,2017-08-05 11:27,True,neural,neural,0.000000,0.500000
3,MY HEART 😭❤️ https://t.co/iKrYLK5apF,2016-03-15 12:21,True,neural,objective,0.000000,0.000000
4,cows are just big grass puppies https://t.co/6...,2017-04-28 20:26,True,negative,objective,-0.066667,0.133333
...,...,...,...,...,...,...,...
54165,RT @spacebongmilk: My girl can be butt ass nak...,2020-07-22 18:19:46,False,neural,objective,0.000000,0.233333
54166,"Not a win again, but people that understands f...",2020-07-23 02:29:33,False,positive,objective,0.200000,0.400000
54167,RT @alizawhoo: @lanascouture A girl can walk o...,2020-07-22 16:56:46,False,positive,objective,0.200000,0.400000
54168,People don't understand what football means to...,2020-07-23 07:04:00,False,negative,objective,-0.206250,0.443750
